In [ ]:
#Different affect and emotion visualisations
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats

In [ ]:
df = pd.read_csv('~/Thesis/WavFiles_extraAudioAnalysis/transcripts/SpeakerTextEmo/speakerEmo.csv')

In [ ]:
df['duration'] = df['End Time'] - df['Start Time']

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(df['Start Time'], df['Arousal'], label='Arousal', color='red')
plt.plot(df['Start Time'], df['Valence'], label='Valence', color='blue')
plt.scatter(df['Start Time'], df['Arousal'], c=df['Speaker'].map({'therapist': 'red', 'patient': 'blue', 'NAN': 'green'}))
plt.scatter(df['Start Time'], df['Valence'], c=df['Speaker'].map({'therapist': 'red', 'patient': 'blue', 'NAN': 'green'}))
plt.xlabel('Time')
plt.ylabel('Emotion Level')
plt.title('Emotion Timeline')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
df_melt = pd.melt(df, id_vars=['Speaker'], value_vars=['Arousal', 'Valence'], var_name='emotion', value_name='level')
sns.boxplot(x='Speaker', y='level', hue='emotion', data=df_melt)
plt.title('Distribution of Emotions by Speaker')
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.imshow([df['approval']], cmap='YlOrRd', aspect='auto')
plt.colorbar(label='Approval Level')
plt.xlabel('Conversation Progress')
plt.title('Approval Heatmap')
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df, x='Arousal', y='Valence', hue='Speaker', size='approval', sizes=(20, 200))
plt.title('Arousal vs Valence')
plt.show()

In [ ]:
window_size = 5  # Adjust as needed
df['arousal_ma'] = df['Arousal'].rolling(window=window_size).mean()
df['valence_ma'] = df['Valence'].rolling(window=window_size).mean()

plt.figure(figsize=(15, 6))
plt.plot(df['Start Time'], df['arousal_ma'], label='Arousal (Moving Avg)', color='red')
plt.plot(df['Start Time'], df['valence_ma'], label='Valence (Moving Avg)', color='blue')
plt.xlabel('Time')
plt.ylabel('Emotion Level')
plt.title(f'Moving Average of Emotions (Window Size: {window_size})')
plt.legend()
plt.show()

In [ ]:
emotions = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

plt.figure(figsize=(15, 8))
plt.stackplot(df['Start Time'], [df[emotion] for emotion in emotions], labels=emotions)
plt.xlabel('Time')
plt.ylabel('Emotion Intensity')
plt.title('Emotion Composition Over Time')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(15, 8))
sns.heatmap(df[emotions].T, cmap='YlOrRd', xticklabels=False)
plt.xlabel('Conversation Progress')
plt.ylabel('Emotions')
plt.title('Emotion Intensity Heatmap')
plt.show()

In [ ]:
def radar_chart(data, categories, title):
    angles = np.linspace(0, 2*np.pi, len(categories), endpoint=False)
    data = np.concatenate((data, [data[0]]))
    angles = np.concatenate((angles, [angles[0]]))
    
    fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(projection='polar'))
    ax.plot(angles, data)
    ax.fill(angles, data, alpha=0.25)
    ax.set_thetagrids(angles[:-1] * 180/np.pi, categories)
    ax.set_title(title)
    plt.show()

therapist_avg = df[df['Speaker'] == 'therapist'][emotions].mean()
patient_avg = df[df['Speaker'] == 'patient'][emotions].mean()

radar_chart(therapist_avg, emotions, 'Average Emotions - Therapist')
radar_chart(patient_avg, emotions, 'Average Emotions - Patient')

In [ ]:
def get_dominant_emotion(row):
    return emotions[row[emotions].argmax()]

df['dominant_emotion'] = df.apply(get_dominant_emotion, axis=1)
transition_matrix = pd.crosstab(df['dominant_emotion'].shift(), df['dominant_emotion'], normalize='index')

plt.figure(figsize=(10, 8))
sns.heatmap(transition_matrix, annot=True, cmap='YlGnBu')
plt.title('Emotion Transition Probabilities')
plt.xlabel('To Emotion')
plt.ylabel('From Emotion')
plt.show()

In [ ]:
correlation_matrix = df[emotions + ['Arousal', 'Valence']].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
plt.title('Correlation between Emotions, Arousal, and Valence')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))

therapist_emotions = df[df['Speaker'] == 'therapist'][emotions].sum()
patient_emotions = df[df['Speaker'] == 'patient'][emotions].sum()

ax1.pie(therapist_emotions, labels=emotions, autopct='%1.1f%%', startangle=90)
ax1.set_title('Therapist Emotion Composition')

ax2.pie(patient_emotions, labels=emotions, autopct='%1.1f%%', startangle=90)
ax2.set_title('Patient Emotion Composition')

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Patient plot
patient_data = df[df['Speaker'] == 'patient']
ax1.plot(patient_data['Start Time'], patient_data['Arousal'], label='Arousal', color='red')
ax1.plot(patient_data['Start Time'], patient_data['Valence'], label='Valence', color='blue')
ax1.set_ylabel('Emotion Level')
ax1.set_title('Patient Arousal and Valence Over Time')
ax1.legend()
ax1.grid(True)

# Therapist plot
therapist_data = df[df['Speaker'] == 'therapist']
ax2.plot(therapist_data['Start Time'], therapist_data['Arousal'], label='Arousal', color='red')
ax2.plot(therapist_data['Start Time'], therapist_data['Valence'], label='Valence', color='blue')
ax2.set_xlabel('Time')
ax2.set_ylabel('Emotion Level')
ax2.set_title('Therapist Arousal and Valence Over Time')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)

# Color palette for emotions
colors = ['red', 'green', 'purple', 'yellow', 'blue', 'orange']

def plot_emotions(ax, data, title):
    # Fill NaN values with 0
    data_filled = data[emotions].fillna(0)
    
    # Create a line plot for each emotion
    for emotion, color in zip(emotions, colors):
        ax.plot(data['Start Time'], data_filled[emotion], color=color, linewidth=2, label=emotion)
    
    ax.set_ylabel('Emotion Intensity')
    ax.set_title(f'{title} Emotions Over Time')
    ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
    ax.set_ylim(bottom=0)  # Ensure the y-axis starts at 0
    ax.grid(True, linestyle='--', alpha=0.7)  # Add a grid for better readability

# Patient plot
patient_data = df[df['Speaker'] == 'patient'].sort_values('Start Time')
plot_emotions(ax1, patient_data, 'Patient')

# Therapist plot
therapist_data = df[df['Speaker'] == 'therapist'].sort_values('Start Time')
plot_emotions(ax2, therapist_data, 'Therapist')

ax2.set_xlabel('Time')

plt.tight_layout()
plt.show()

In [ ]:
colors = {'anger': 'red', 'disgust': 'green', 'fear': 'purple', 
          'joy': 'yellow', 'sadness': 'blue', 'surprise': 'orange'}

def plot_affect_emotions(data, title_prefix):
    # Fill NaN values with 0
    data_filled = data[emotions + ['Arousal', 'Valence']].fillna(0)
    
    # 1. Circumplex model
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Normalize arousal and valence to range from -1 to 1
    arousal = 2 * ((data_filled['Arousal'] - data_filled['Arousal'].min()) / (data_filled['Arousal'].max() - data_filled['Arousal'].min())) - 1
    valence = 2 * ((data_filled['Valence'] - data_filled['Valence'].min()) / (data_filled['Valence'].max() - data_filled['Valence'].min())) - 1
    
    scatter = ax.scatter(valence, arousal, alpha=0.5, c=data['Start Time'], cmap='viridis')
    
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label('Time Progression', rotation=270, labelpad=15)

    circle1 = plt.Circle((0, 0), 0.5, fill=False, color='gray')
    circle2 = plt.Circle((0, 0), 1, fill=False, color='gray')
    ax.add_artist(circle1)
    ax.add_artist(circle2)

    ax.axhline(y=0, color='k', linestyle='--')
    ax.axvline(x=0, color='k', linestyle='--')

    ax.set_xlim(-1.1, 1.1)
    ax.set_ylim(-1.1, 1.1)
    ax.set_xlabel('Valence', fontsize=12)
    ax.set_ylabel('Arousal', fontsize=12)
    ax.set_title(f'{title_prefix} Arousal-Valence Circumplex', fontsize=14)

    ax.text(0.85, 0.85, 'High Arousal\nPositive Valence', ha='center', va='center', fontsize=10)
    ax.text(-0.85, 0.85, 'High Arousal\nNegative Valence', ha='center', va='center', fontsize=10)
    ax.text(-0.85, -0.85, 'Low Arousal\nNegative Valence', ha='center', va='center', fontsize=10)
    ax.text(0.85, -0.85, 'Low Arousal\nPositive Valence', ha='center', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

    # 2. Time-based plots
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
    
    # Arousal and valence over time
    ax1.plot(data['Start Time'], data_filled['Arousal'], color='red', label='Arousal')
    ax1.plot(data['Start Time'], data_filled['Valence'], color='blue', label='Valence')
    ax1.set_ylabel('Affect Level')
    ax1.set_title(f'{title_prefix} Arousal and Valence Over Time')
    ax1.legend(loc='upper left')
    
    # Emotions over time
    for emotion, color in colors.items():
        ax2.plot(data['Start Time'], data_filled[emotion], color=color, label=emotion)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Emotion Intensity')
    ax2.set_title(f'{title_prefix} Emotions Over Time')
    ax2.legend(loc='upper left')

    plt.tight_layout()
    plt.show()

    # 3. Correlation heatmap
    fig, ax = plt.subplots(figsize=(10, 8))
    corr = data_filled[emotions + ['Arousal', 'Valence']].corr()
    sns.heatmap(corr, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0, ax=ax)
    ax.set_title(f'{title_prefix} Correlation between Emotions, Arousal, and Valence')
    plt.tight_layout()
    plt.show()


# Create plots for patient and therapist
plot_affect_emotions(df[df['Speaker'] == 'patient'], 'Patient Affect and Emotions')
plot_affect_emotions(df[df['Speaker'] == 'therapist'], 'Therapist Affect and Emotions')

In [ ]:
def plot_affect_emotions(df):
    # Fill NaN values with 0
    df_filled = df[emotions + ['Arousal', 'Valence', 'Speaker', 'Start Time']].fillna(0)
    
    # 1. Circumplex model
    fig, ax = plt.subplots(figsize=(12, 12))
    
    scatter = ax.scatter(df_filled['Valence'], df_filled['Arousal'], alpha=0.5, c=df_filled['Start Time'], cmap='viridis')
    
    cbar = fig.colorbar(scatter, ax=ax)
    cbar.set_label('Time Progression', rotation=270, labelpad=15)

    circle1 = plt.Circle((0.5, 0.5), 0.25, fill=False, color='gray')
    circle2 = plt.Circle((0.5, 0.5), 0.5, fill=False, color='gray')
    ax.add_artist(circle1)
    ax.add_artist(circle2)

    ax.axhline(y=0.5, color='k', linestyle='--')
    ax.axvline(x=0.5, color='k', linestyle='--')

    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xlabel('Valence', fontsize=12)
    ax.set_ylabel('Arousal', fontsize=12)
    ax.set_title('Arousal-Valence Circumplex', fontsize=14)

    ax.text(0.85, 0.85, 'High Arousal\nPositive Valence', ha='center', va='center', fontsize=10)
    ax.text(0.15, 0.85, 'High Arousal\nNegative Valence', ha='center', va='center', fontsize=10)
    ax.text(0.15, 0.15, 'Low Arousal\nNegative Valence', ha='center', va='center', fontsize=10)
    ax.text(0.85, 0.15, 'Low Arousal\nPositive Valence', ha='center', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

    # 2. Valence and Arousal over time for Therapist and Patient
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
    
    therapist_data = df_filled[df_filled['Speaker'] == 'therapist']
    patient_data = df_filled[df_filled['Speaker'] == 'patient']
    
    ax1.plot(therapist_data['Start Time'], therapist_data['Arousal'], color='red', label='Arousal')
    ax1.plot(therapist_data['Start Time'], therapist_data['Valence'], color='blue', label='Valence')
    ax1.set_ylabel('Affect Level')
    ax1.set_title('Therapist Arousal and Valence Over Time')
    ax1.legend()
    
    ax2.plot(patient_data['Start Time'], patient_data['Arousal'], color='red', label='Arousal')
    ax2.plot(patient_data['Start Time'], patient_data['Valence'], color='blue', label='Valence')
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Affect Level')
    ax2.set_title('Patient Arousal and Valence Over Time')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

    # 3. Emotions over time for Therapist and Patient
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 12), sharex=True)
    
    for emotion, color in colors.items():
        ax1.plot(therapist_data['Start Time'], therapist_data[emotion], color=color, label=emotion)
    ax1.set_ylabel('Emotion Intensity')
    ax1.set_title('Therapist Emotion Count Over Time')
    ax1.legend()
    
    for emotion, color in colors.items():
        ax2.plot(patient_data['Start Time'], patient_data[emotion], color=color, label=emotion)
    ax2.set_xlabel('Time')
    ax2.set_ylabel('Emotion Count')
    ax2.set_title('Patient Emotion Count Over Time')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()

    # 4. Distribution of Affect by Speaker
    plt.figure(figsize=(10, 6))
    df_melt = pd.melt(df_filled, id_vars=['Speaker'], value_vars=['Arousal', 'Valence'], var_name='affect', value_name='level')
    sns.boxplot(x='Speaker', y='level', hue='affect', data=df_melt)
    plt.title('Distribution of Affect by Speaker')
    plt.show()

    # 5. Emotion Composition Pie Charts
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    therapist_emotions = df_filled[df_filled['Speaker'] == 'therapist'][emotions].sum()
    patient_emotions = df_filled[df_filled['Speaker'] == 'patient'][emotions].sum()
    ax1.pie(therapist_emotions, labels=emotions, autopct='%1.1f%%', startangle=90, colors=colors.values())
    ax1.set_title('Therapist Emotion Composition')
    ax2.pie(patient_emotions, labels=emotions, autopct='%1.1f%%', startangle=90, colors=colors.values())
    ax2.set_title('Patient Emotion Composition')
    plt.tight_layout()
    plt.show()

# Create plots
plot_affect_emotions(df)